In [1]:
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_wine,load_breast_cancer,load_diabetes, make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold, ParameterGrid
from sklearn.metrics import accuracy_score

from bayesian_hpo import find_best_hyperparams


import warnings
warnings.filterwarnings('ignore')

# toy datasets: wine, breast_cancer, diabetes
# real datasets: california_housing
# synthetic datasets: make_classification

# try random forest + regression task


## TODO: scaled performance comparison

In [2]:
#data = load_breast_cancer()
#X, y = data['data'], data['target']

X, y = make_classification(n_samples=1000,n_features=32,n_informative=20,n_classes=4)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### Bayesian HPO

In [6]:
# inputs
param_grid = {'C': np.logspace(-4, 4, 20), 'l1_ratio': np.arange(0,1.01,0.1), 'fit_intercept':[True,False]}
categorical_feats = ['fit_intercept'] #boolean values as well

model = LogisticRegression(solver = 'saga', penalty='elasticnet')
metric_fnc = accuracy_score

find_best_hyperparams(X_train,y_train,param_grid,categorical_feats,model,metric_fnc)

hyper_config: {'C': 78.47599703514607, 'fit_intercept': True, 'l1_ratio': 0.2}, mean: 0.5403, std: 0.0571


### Grid Search

In [7]:
print(f'size of hyperparameter space: {len(list(ParameterGrid(param_grid)))}')

scaler = StandardScaler()
kf = KFold(n_splits=5)

best_score = -1
best_score_std = None
best_config = None

for hyp_cfg in list(ParameterGrid(param_grid)):

    scores = []
    for train_index, test_index in kf.split(X_train):   
        X_train_cv, X_valid_cv = X_train[train_index], X_train[test_index]
        y_train_cv, y_valid_cv = y_train[train_index], y_train[test_index]

        X_train_cv = scaler.fit_transform(X_train_cv)
        X_valid_cv = scaler.transform(X_valid_cv)

        model = LogisticRegression(solver = 'saga', penalty='elasticnet',**hyp_cfg)

        model.fit(X_train_cv,y_train_cv)

        y_pred_cv = model.predict(X_valid_cv)
        scores.append(accuracy_score(y_valid_cv,y_pred_cv))

    if best_score < np.mean(scores):
        best_score = np.mean(scores)
        best_score_std = np.std(scores)
        best_config = hyp_cfg


print(f'hyper_config: {best_config}, mean: {best_score:.4f}, std: {best_score_std:.4f}')

size of hyperparameter space: 440
hyper_config: {'C': 0.615848211066026, 'fit_intercept': True, 'l1_ratio': 0.9}, mean: 0.5448, std: 0.0610
